In [70]:
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [2]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
#import pandas as pd
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import conllu

In [36]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  #database="small_commits_db"
  database='commits_db',
  charset='utf8mb4')

mycursor = mydb.cursor()

## Data by time

In [62]:
def yield_lemma(year, month, mode='exact'):
    dt = str(year)+'-'+month
    if mode == 'exact':
        mycursor.execute('SELECT conllu FROM C WHERE date LIKE "{}%"'.format(dt))
    elif mode == 'before':
        mycursor.execute('SELECT conllu FROM C WHERE date < %s', (str(year),))
    else:
        mycursor.execute('SELECT conllu FROM C WHERE date >= %s', (str(year),))
    text = mycursor.fetchone()
    #print (text)
    while text:
        doc = conllu.parse(text[0])
        for sent in doc:
            snt = []
            for i in sent.tokens:
                snt.append(i['lemma'])
            yield snt
        text = mycursor.fetchone()

In [66]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  #database="small_commits_db"
  database='commits_db',
  charset='utf8mb4')

mycursor = mydb.cursor()

In [67]:
with open('../data/by_month/1999-before.txt', 'w') as file:
    for item in tqdm(yield_lemma(2000, '00', mode='before')):
        sent = item
        print(' '.join(sent).strip(), file=file)

In [68]:
with open('../data/by_month/2019-after.txt', 'w') as file:
    for item in tqdm(yield_lemma(2019, '00', mode='after')):
        sent = item
        print(' '.join(sent).strip(), file=file)

In [69]:
a = 2000
b = 2019
x = ['01', '02', '03', '04','05','06', '07','08','09', '10','11','12']

In [ ]:
for year in tqdm(range(a, b)):
    for month in tqdm(x, leave=False):
        with open('../data/by_month/{}-{}.txt'.format(str(year), month), 'w') as file:
            for item in tqdm(yield_lemma(year, month, mode='exact'), leave=False):
                sent = item
                print(' '.join(sent).strip(), file=file)

In [ ]:
date = None
for item in tqdm(yield_lemma()):
    date1, sent = item
    print(date1, sent)
    if date1 != date:
        date = date1
        file = open('../data/by_month/{}.txt'.format(date), 'w')
    print(' '.join(sent).strip(), file=file)

## Frequency dict

In [12]:
! wc -l frequency_dict_no_pos_proc.csv

113983 frequency_dict_no_pos_proc.csv


In [14]:
! tail -n 5 frequency_dict_no_pos_proc.csv

decisiontable	0	0	0	0	0	0	101	0	0	0	0	0	0	0	0	0	101
19955	12	5	15	0	2	1	1	2	0	31	5	7	0	0	3	17	101
webtextrenderer	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	101	101
37051	4	0	74	0	2	0	1	0	0	7	1	7	1	0	0	4	101
rmlock	100	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	101


In [25]:
freq_dict = {}
k = 0
for line in open('frequency_dict_no_pos_proc.csv'):
    if k == 0:
        k += 1
    else:
        row = line.strip().split('\t')
        if row[0].isalpha():
            freq_dict[row[0]] = int(row[-1])

## W2V time

In [71]:
w2v = Word2Vec()

In [72]:
w2v.build_vocab_from_freq(freq_dict, update=False)

In [79]:
k = 0
w = 0
for line in  open('../data/by_month/1999-before.txt'):
    w += len(line.split())
    k += 1
print(k, w)
w2v.train(total_examples=k, total_words = w, corpus_file='../data/by_month/1999-before.txt', epochs=10)

1178804 10423409


(42643705, 103319350)

In [80]:
w2v.wv.most_similar('fix')

[('fixe', 0.7021887302398682),
 ('fixed', 0.7015795707702637),
 ('correct', 0.651726484298706),
 ('repair', 0.6419893503189087),
 ('cure', 0.596564769744873),
 ('fixing', 0.5345549583435059),
 ('nasty', 0.5187071561813354),
 ('uncover', 0.513318657875061),
 ('solve', 0.5057026743888855),
 ('involve', 0.4987739324569702)]

In [85]:
import os
from collections import defaultdict
lsr = os.listdir('../data/by_month/')
yrs = defaultdict(list)
for i in lsr:
    yrs[i[:4]].append('../data/by_month/'+i)
yrs = dict(yrs)

In [86]:
yrs

{'1999': ['../data/by_month/1999-before.txt'],
 '2019': ['../data/by_month/2019-after.txt']}

In [88]:
for year in tqdm(yrs):
    fname = '../data/by_month_vecs/'+str(year)+'.txt'
    for month in tqdm(yrs[year], leave=False):
        k = 0
        w = 0
        for line in tqdm(open(month), leave=False):
            w += len(line.split())
            k += 1
        w2v.train(total_examples=k, total_words = w, corpus_file=month, epochs=10)
    w2v.wv.save_word2vec_format(fname, binary=False)